# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [29]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [30]:
' '.join(whitespace_string.strip().split()).replace('a lot of', 'keine')

'This is a string that has keine extra whitespace.'

### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [31]:
# tools for the job
import pandas as pd
import re as rx
import requests

s = requests.get('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt')

In [32]:
no_n = s.text.split('\n')
no_r = [s.replace('\r', '') for s in no_n]

In [33]:
def clean(raw):
    m = [rx.findall(r'[A-Z][a-z]+', i)[0] for i in raw]
    d = [rx.findall(r'[\d]{1,2}', i)[0] for i in raw]
    y = [rx.findall(r'[\d]{4}', i)[0] for i in raw]
    
    df_dates = pd.DataFrame({'Month' : m,'Day' : d,'Year' : y})
    
    return df_dates

clean(no_r)

,Month,Day,Year
0,March,8,2015
1,March,15,2015
2,March,22,2015
3,March,29,2015
4,April,5,2015
5,April,12,2015
6,April,19,2015
7,April,26,2015
8,May,3,2015
9,May,10,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [34]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lambda_school_loaner_32/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
raw = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv')
raw.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [36]:
df = raw.copy()
df.columns = ['sen', 'sen_text']

In [37]:
df.head()

,sen,sen_text
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


## Somewhat inelegant clean

In [38]:
#tokenize descriptions
df.sen_text = df.sen_text.apply(word_tokenize)

# convert to lowercase
df.sen_text = [[el.lower() for el in row] for row in df.sen_text]

# remove punctuation
table = str.maketrans('','', string.punctuation)
df.sen_text = [[x.translate(table) for x in row] for row in df.sen_text]

# filter for alpha characters
df.sen_text = [[el for el in row if el.isalpha()] for row in df.sen_text]

# remove stop words
df.sen_text = [[el for el in row if not el in stop_words] for row in df.sen_text]

In [39]:
for i in range(10):
    print(df.sen_text[(i*10)])

['sad', 'apl', 'friend']
['must', 'think', 'positive']
['sad', 'iran']
['hate', 'athlete', 'appears', 'tear', 'acl', 'live', 'television']
['amp', 'amp', 'fightiin', 'wiit', 'babes']
['baddest', 'day', 'eveer']
['hate', 'u', 'leysh']
['never', 'thought', 'become', 'second', 'choice']
['send', 'sunshine', 'northern', 'ireland', 'going', 'swimming', 'today', 'kezbat']
['jonas', 'day', 'almost']


### How should TF-IDF scores be interpreted? How are they calculated?

**TF-IDF** is all in the name: **_Term Frequency, Inverse Document Frequency_**.  These scores try and highlight how important certain terms are to **particular** documents of a corpus.  If a term occurs a lot accross documents (some words just occur more often!) it is penalized heavily while terms that occur less frequently accross a body of documents gain a slight boost.

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [40]:
# processing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# modeling
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [44]:
# was getting an error having these as arrays so this converts back to strings for modeling.
patch = [' '.join(i) for i in df.sen_text]
df.assign(st = patch);

,sen,sen_text,st
0,0,"[sad, apl, friend]",sad apl friend
1,0,"[missed, new, moon, trailer]",missed new moon trailer
2,1,"[omg, already]",omg already
3,0,"[omgaga, im, sooo, im, gunna, cry, dentist, si...",omgaga im sooo im gunna cry dentist since supo...
4,0,"[think, mi, bf, cheating, tt]",think mi bf cheating tt
5,0,"[worry, much]",worry much
6,1,"[juuuuuuuuuuuuuuuuussssst, chillin]",juuuuuuuuuuuuuuuuussssst chillin
7,0,"[sunny, work, tomorrow, tv, tonight]",sunny work tomorrow tv tonight
8,1,"[handed, uniform, today, miss, already]",handed uniform today miss already
9,1,"[hmmmm, wonder, number]",hmmmm wonder number


In [45]:
# data/target assignment
X = df.st
y = df.sen

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## TFIDF + Logistic

In [21]:
params =  {'tfidfvectorizer__ngram_range' : [(1,1), (1,2), (1,3)],
           'tfidfvectorizer__max_features' : [50, 100, None],
          }

pipe = make_pipeline(TfidfVectorizer(),
                     LogisticRegression(solver='lbfgs',
                     max_iter=200))

log_grid_cv = GridSearchCV(pipe, params, cv=3, scoring='roc_auc')
log_grid_cv.fit(X_train, y_train);

In [22]:
print (log_grid_cv.best_params_)

print(f'Train ROCAUC: {roc_auc_score(log_grid_cv.predict(X_train), y_train):.2f}')
print(f'Test ROCAUC: {roc_auc_score(log_grid_cv.predict(X_test), y_test):.2f}') 

{'tfidfvectorizer__max_features': None, 'tfidfvectorizer__ngram_range': (1, 2)}
Train ROCAUC: 0.89
Test ROCAUC: 0.76


## TFIDF + Naive Bayes

In [23]:
#using the best params from the first tfidf
params =  {'tfidfvectorizer__ngram_range' : [(1,2)],
           'tfidfvectorizer__max_features' : [None],
          }

pipe = make_pipeline(TfidfVectorizer(),
                     MultinomialNB())

nb_grid_cv = GridSearchCV(pipe, params, cv=3, scoring='roc_auc')
nb_grid_cv.fit(X_train, y_train);

In [24]:
print(f'Train ROCAUC: {roc_auc_score(nb_grid_cv.predict(X_train), y_train):.2f}')
print(f'Test ROCAUC: {roc_auc_score(nb_grid_cv.predict(X_test), y_test):.2f}') 

Train ROCAUC: 0.96
Test ROCAUC: 0.76


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [25]:
from gensim.models.word2vec import Word2Vec

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [46]:
model = Word2Vec(df.sen_text, window=3, size=500)
list(model.wv.vocab)

['sad',
 'friend',
 'missed',
 'new',
 'moon',
 'trailer',
 'omg',
 'already',
 'im',
 'sooo',
 'gunna',
 'cry',
 'dentist',
 'since',
 'get',
 'crown',
 'put',
 'think',
 'mi',
 'bf',
 'cheating',
 'tt',
 'worry',
 'much',
 'chillin',
 'sunny',
 'work',
 'tomorrow',
 'tv',
 'tonight',
 'handed',
 'uniform',
 'today',
 'miss',
 'hmmmm',
 'wonder',
 'number',
 'must',
 'positive',
 'thanks',
 'haters',
 'face',
 'day',
 'weekend',
 'sucked',
 'far',
 'jb',
 'isnt',
 'showing',
 'australia',
 'ok',
 'thats',
 'win',
 'lt',
 'way',
 'feel',
 'right',
 'man',
 'completely',
 'useless',
 'rt',
 'funny',
 'twitter',
 'http',
 'feeling',
 'strangely',
 'fine',
 'gon',
 'na',
 'go',
 'listen',
 'celebrate',
 'huge',
 'roll',
 'thunder',
 'scary',
 'cut',
 'beard',
 'growing',
 'well',
 'year',
 'start',
 'happy',
 'meantime',
 'iran',
 'one',
 'see',
 'cause',
 'else',
 'following',
 'pretty',
 'awesome',
 'level',
 'writing',
 'massive',
 'blog',
 'tweet',
 'myspace',
 'comp',
 'shut',
 'lost

In [47]:
model.wv.most_similar('twitter', topn=5)

[('email', 0.8626171350479126),
 ('facebook', 0.8599664568901062),
 ('link', 0.8536689281463623),
 ('dm', 0.8509138822555542),
 ('list', 0.837100088596344)]

In [48]:
#hmmmm
model.wv.most_similar('drunk', topn=5)

[('memo', 0.9879697561264038),
 ('whining', 0.9863419532775879),
 ('aots', 0.9845591187477112),
 ('hubby', 0.98436039686203),
 ('starving', 0.9843247532844543)]

In [49]:
model.wv.most_similar('swimming', topn=5)

[('parents', 0.9831657409667969),
 ('dad', 0.983155369758606),
 ('focused', 0.9827832579612732),
 ('warped', 0.9826194047927856),
 ('noon', 0.9817023277282715)]